In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import torch
import torch.nn as nn
from timm import create_model
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import numpy as np
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

import random
import numpy as np

def seed_everything(seed=28):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(28)

# ✅ 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_root = f'../data/train2'
test_csv_path = '../data/test.csv'
sample_sub_path = '../data/sample_submission.csv'
save_dir = '../submissions'
base_save_dir = '../checkpoints'
num_epochs = 45
batch_size = 32

best_model_name = 'swin_tiny_patch4_window7_224'

# ✅ 전체 데이터로 재학습
from dataloaders.loaders import get_dataloaders
from models.train_utils import train_one_epoch

dataloaders = get_dataloaders(data_root, batch_size=batch_size, val_ratio=0.0, return_path=False)
train_loader = dataloaders[best_model_name]['train']
class_names = dataloaders[best_model_name]['classes']
num_classes = len(class_names)

model = create_model(best_model_name, pretrained=True, num_classes=num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

print("🔁 전체 학습 데이터로 재학습 시작")
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    print(f"📘 Epoch {epoch + 1} 🔹 Training Loss: {train_loss:.4f} 🔹 Learning Rate: {scheduler.get_last_lr()[0]:.6f}")
    scheduler.step()

# ✅ 모델 저장
os.makedirs(base_save_dir, exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_path = os.path.join(base_save_dir, f'{best_model_name}_{timestamp}.pth')
torch.save(model.state_dict(), model_path)
print(f"💾 모델 저장 완료: {model_path}")

model.eval()

# ✅ 테스트 전처리
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ✅ 테스트 데이터 로드
test_df = pd.read_csv(test_csv_path)
submission_template = pd.read_csv(sample_sub_path)
submission_class_names = submission_template.columns.tolist()[1:]

# ✅ 예측
results = []
with torch.no_grad():
    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc='🔍 테스트셋 예측'):
        img_path = os.path.join('../data', row['img_path'])
        image = Image.open(img_path).convert('RGB')

        input_tensor = transform(image).unsqueeze(0).to(device)
        output = model(input_tensor)
        prob = torch.softmax(output, dim=1).squeeze().cpu().numpy()
        results.append(prob)

# ✅ 제출 파일 생성
submission_df = pd.DataFrame(results, columns=class_names)
submission_df.insert(0, 'ID', test_df['ID'])

os.makedirs(save_dir, exist_ok=True)
submission_path = os.path.join(save_dir, f'submission_{best_model_name}_{timestamp}.csv')
submission_df.to_csv(submission_path, index=False)

print(f"✅ 제출 파일 저장 완료: {submission_path}\n")


🔁 전체 학습 데이터로 재학습 시작


📘 Epoch 1 🔹 Training Loss: 3.8719 🔹 Learning Rate: 0.000100


📘 Epoch 2 🔹 Training Loss: 1.6973 🔹 Learning Rate: 0.000100


📘 Epoch 3 🔹 Training Loss: 1.2012 🔹 Learning Rate: 0.000100


📘 Epoch 4 🔹 Training Loss: 0.9646 🔹 Learning Rate: 0.000099


...

📘 Epoch 48 🔹 Training Loss: 0.1351 🔹 Learning Rate: 0.000001


📘 Epoch 49 🔹 Training Loss: 0.1416 🔹 Learning Rate: 0.000000


📘 Epoch 50 🔹 Training Loss: 0.1350 🔹 Learning Rate: 0.000000
💾 모델 저장 완료: ../checkpoints\swin_tiny_patch4_window7_224_20250528_023613.pth


🔍 테스트셋 예측: 100%|██████████| 8258/8258 [04:00<00:00, 34.33it/s]


✅ 제출 파일 저장 완료: ../submissions\submission_swin_tiny_patch4_window7_224_20250528_023613.csv



In [1]:
# import sys
# import os
# sys.path.append(os.path.abspath('..'))

# import pandas as pd
# import torch
# import torch.nn as nn
# from timm import create_model
# from torchvision import transforms
# from PIL import Image
# from tqdm import tqdm
# import numpy as np
# import warnings
# from datetime import datetime
# warnings.filterwarnings('ignore')

# import random
# import numpy as np

# def seed_everything(seed=28):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# seed_everything(28)

# # ✅ 설정
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# data_root = f'../data/train2'
# test_csv_path = '../data/test.csv'
# sample_sub_path = '../data/sample_submission.csv'
# save_dir = '../submissions'
# base_save_dir = '../checkpoints'  # 🔸 모델 저장 경로 추가
# num_epochs = 50
# patience = 3
# batch_size = 32

# best_model_name = 'vit_base_patch16_224'

# # ✅ 전체 데이터로 재학습
# from dataloaders.loaders import get_dataloaders
# from models.train_utils import train_one_epoch, EarlyStopping

# dataloaders = get_dataloaders(data_root, batch_size=batch_size, val_ratio=0.0, return_path=False)
# train_loader = dataloaders[best_model_name]['train']
# class_names = dataloaders[best_model_name]['classes']
# num_classes = len(class_names)

# model = create_model(best_model_name, pretrained=True, num_classes=num_classes).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# criterion = nn.CrossEntropyLoss()
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
# early_stopping = EarlyStopping(patience=patience)

# print("🔁 전체 학습 데이터로 재학습 시작")
# for epoch in range(num_epochs):
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
#     print(f"📘 Epoch {epoch + 1} 🔹 Training Loss: {train_loss:.4f} 🔹 Learning Rate: {scheduler.get_last_lr()[0]:.6f}")
#     scheduler.step()
#     early_stopping(train_loss, model)
#     if early_stopping.early_stop:
#         print("⛔ Early stopping 발생")
#         break

# # ✅ 모델 저장
# model.load_state_dict(early_stopping.best_model_state)
# os.makedirs(base_save_dir, exist_ok=True)
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# model_path = os.path.join(base_save_dir, f'{best_model_name}_{timestamp}.pth')
# torch.save(model.state_dict(), model_path)
# print(f"💾 모델 저장 완료: {model_path}")

# model.eval()

# # ✅ 테스트 전처리
# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
# ])

# # ✅ 테스트 데이터 로드
# test_df = pd.read_csv(test_csv_path)
# submission_template = pd.read_csv(sample_sub_path)
# submission_class_names = submission_template.columns.tolist()[1:]

# # ✅ 예측
# results = []
# with torch.no_grad():
#     for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc='🔍 테스트셋 예측'):
#         img_path = os.path.join('../data', row['img_path'])
#         image = Image.open(img_path).convert('RGB')

#         input_tensor = transform(image).unsqueeze(0).to(device)
#         output = model(input_tensor)
#         prob = torch.softmax(output, dim=1).squeeze().cpu().numpy()
#         results.append(prob)

# # ✅ 제출 파일 생성
# submission_df = pd.DataFrame(results, columns=class_names)
# submission_df.insert(0, 'ID', test_df['ID'])

# os.makedirs(save_dir, exist_ok=True)
# submission_path = os.path.join(save_dir, f'submission_{best_model_name}_{timestamp}.csv')
# submission_df.to_csv(submission_path, index=False)

# print(f"✅ 제출 파일 저장 완료: {submission_path}\n")

# import sys
# import os
# sys.path.append(os.path.abspath('..'))  # 상위 폴더 추가

# import pandas as pd
# import torch
# import torch.nn as nn
# from timm import create_model
# from torchvision import transforms
# from PIL import Image
# from tqdm import tqdm
# import numpy as np
# import warnings
# from datetime import datetime
# warnings.filterwarnings('ignore')

# import random
# import numpy as np

# def seed_everything(seed=28):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)

#     # cuDNN 연산 결정적 설정
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False


# seed_everything(28)

# # ✅ 설정
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# data_root = f'../data/train2'
# test_csv_path = '../data/test.csv'
# sample_sub_path = '../data/sample_submission.csv'
# save_dir = '../submissions'
# num_epochs = 10
# patience = 3
# batch_size = 32

# # 최고 성능 모델을 설정해주세요.
# best_model_name = 'vit_base_patch16_224'

# # ✅ 전체 데이터로 재학습
# from dataloaders.loaders import get_dataloaders
# from models.train_utils import train_one_epoch, EarlyStopping

# dataloaders = get_dataloaders(data_root, batch_size=batch_size, val_ratio=0.0, return_path=False)
# train_loader = dataloaders[best_model_name]['train']
# class_names = dataloaders[best_model_name]['classes']
# num_classes = len(class_names)

# model = create_model(best_model_name, pretrained=True, num_classes=num_classes).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# criterion = nn.CrossEntropyLoss()
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
# early_stopping = EarlyStopping(patience=patience)

# print("🔁 전체 학습 데이터로 재학습 시작")
# for epoch in range(num_epochs):
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
#     print(f"📘 Epoch {epoch + 1}")
#     print(f"   🔹 Training Loss: {train_loss:.4f}")
#     scheduler.step()
#     early_stopping(train_loss, model)
#     if early_stopping.early_stop:
#         print("⛔ Early stopping 발생")
#         break

# model.load_state_dict(early_stopping.best_model_state)
# model.eval()

# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
# ])

# # ✅ 테스트 데이터 로드
# test_df = pd.read_csv(test_csv_path)
# submission_template = pd.read_csv(sample_sub_path)
# submission_class_names = submission_template.columns.tolist()[1:]

# # ✅ 예측
# results = []
# with torch.no_grad():
#     for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc='🔍 테스트셋 예측'):
#         img_path = os.path.join('../data', row['img_path'])
#         image = Image.open(img_path).convert('RGB')

#         input_tensor = transform(image).unsqueeze(0).to(device)
#         output = model(input_tensor)
#         prob = torch.softmax(output, dim=1).squeeze().cpu().numpy()
#         results.append(prob)

# # ✅ 제출 파일 생성
# submission_df = pd.DataFrame(results, columns=class_names)
# submission_df.insert(0, 'ID', test_df['ID'])

# os.makedirs(save_dir, exist_ok=True)
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# submission_path = os.path.join(save_dir, f'submission_{best_model_name}_{timestamp}.csv')
# submission_df.to_csv(submission_path, index=False)

# print(f"✅ 제출 파일 저장 완료: {submission_path}\n")


🔁 전체 학습 데이터로 재학습 시작


📘 Epoch 1
   🔹 Training Loss: 5.8434


📘 Epoch 2
   🔹 Training Loss: 3.8131


📘 Epoch 3
   🔹 Training Loss: 1.8237


📘 Epoch 4
   🔹 Training Loss: 1.1614


...

📘 Epoch 3
   🔹 Training Loss: 3.1375


📘 Epoch 4
   🔹 Training Loss: 1.7895


📘 Epoch 5
   🔹 Training Loss: 1.2885


🔍 테스트셋 예측: 100%|██████████| 8258/8258 [02:17<00:00, 60.20it/s]


✅ 제출 파일 저장 완료: ../submissions\submission_vit_base_patch16_224_20250524_150508.csv
